## Anwendung: Logistische Regression

In diesem Notebook trainieren wir ein Modell, das für den Kunden einer Website vorhersagen soll, ob er oder sie ein Produkt kauft oder nicht. Grundlage für dieses Klassifikationsproblem sind demografische Daten sowie das Verhalten der Kunden. Angenommen, wir erheben für die Kunden folgende Merkmale:

- Age: Alter des Kunden ($x_1$)
- Gender: Geschlecht des Kunden, wobei 0=weiblich, 1=männlich ($x_2$)
- Annual Income: Jahreseinkommen in tausend Euro ($x_3$)
- Browsing Time: Verbrachte Zeit auf der Website in Minuten ($x_4$)
- Previous Purchases: Anzahl vorheriger Käufe des Kunden ($x_5$)

Wir haben von 100 Kunden folgende Daten gesammelt:

In [3]:
import pandas as pd

df = pd.read_csv("Daten/purchase.csv")
df

,Age,Gender,Annual Income,Browsing Time,Previous Purchases,Purchased
0,22,0,40,5,0,0
1,35,1,60,10,3,1
2,26,1,30,3,1,0
3,29,0,55,8,2,1
4,42,1,80,12,4,1
...,...,...,...,...,...,...
95,24,1,34,3,1,0
96,37,1,81,13,4,1
97,28,0,40,5,1,0
98,33,1,64,9,3,1


Die Spalte `Purchased` gibt an, ob der Kunde das Produkt gekauft hat. Im maschinellen Lernen bezeichnet man dies als Label, welches meist mit $y_i, i=1,\dots,100$ bezeichnet wird. 

Wir trainieren nun ein logistisches Regressionsmodell, welches die Spalte `Purchased` möglichst gut aus den anderen vorhersagen soll. Dazu machen wir folgenden Ansatz für die Modellfunktion:

\begin{align*}
    P(\texttt{Purchased}) = \sigma(\v w^T\v x) = \frac{1}{1+e^{-\v w^T \v x}} = \frac{1}{1+e^{-(w_1x_1 + w_2x_2 + w_3x_3 + w_4x_4 + w_5x_5)}},
\end{align*}

Die Funktion 
\begin{align*}
    \sigma(t) = \frac{1}{1+e^{-t}}
\end{align*}
ist die *logistische Funktion*. Diese stellt sicher, dass die Vorhersage zwischen $0$ und $1$ liegt und als Wahrscheinlichkeit interpretiert werden kann.

Aufgabe des Modelltrainings ist es nun, diejenigen Parameter $w_1,\dots, w_5$ zu identifizieren, so dass die vorhergesagten Wahrscheinlichkeiten $f(\v x; \v w)$ möglichst gut zu den echten Werten der Spalte `Purchased` passen. Für die lineare Regression misst man dies mit dem mittleren quadratischen Fehler. Dies ist eine quadratische Funktion, für die man (über die Normalengleichungen) ein globales Minimum analytisch bestimmen kann. Für die logistische Regression nimmt man stattdessen die binäre Kreuzentropie:

\begin{align*}
 L(\v w)=-\frac{1}{N}\sum_{i=1}^{100} \ln{\left[ y_i \cdot \sigma(\v w^T\v x_i) + (1-y_i) \cdot (1-\sigma(\v w^T\v x_i))\right]}  
\end{align*}

Diese Funktion gilt es nun zu minimieren. Ihr Minimum kann im Gegensatz zur linearen Regression nicht mehr analytisch bestimmt werden, sondern es werden iterative Verfahren benötigt (Gradientenverfahren bzw. Newtonverfahren).

Wir definieren dazu zunächst den Labelvektor $\v y$ sowie die Designmatrix $\v X$. Wir casten sowohl $\v y$ und $\v X$ explizit als `autograd.numpy` arrays, damit die `autograd` diese beim Differenzieren korrekt verarbeiten kann. Anschließend definieren wir die logistische Funktion sowie die binäre Kreuzentropie als Python Funktionen. 

### Gradientenverfahren

In [4]:
import autograd.numpy as np

# Labelvektor
y = np.array(df.Purchased)

# Designmatrix
X = df.drop("Purchased", axis=1)
X = (X - X.mean()) / X.std()

# Explizit als autograd numpy array casten
X = np.array(X)

# Logistische Funktion
def logistic(x, w):
    return 1.0 / (1 + np.exp(-x@w))

# Binäre Kreuzentropie
def binary_cross_entropy(w):
    
    y_pred = logistic(X,w)

    return -np.sum(np.log(y*y_pred + (1-y)*(1-y_pred)))

Wir benutzen ein Gradientenverfahren mit fester Schrittweite um die optimalen Parameter zu berechnen. Dank `autograd` müssen wir uns um die Berechnung der Ableitung keine Gedanken machen. Wir initialisieren alle Gewichte $w_1,\dots,w_5$ mit $0$, d.h. $\v w^{[0]}=\v 0$. Das ist gleichbedeutend mit einer Vorhersage von 50% Kaufwahrscheinlichkeit für jeden Kunden.

In [5]:
from autograd import grad

def gd(func, alpha, w0, n_steps=10000):
    """ Perform n_steps iterations of gradient descent with steplength alpha and return iterates """
    w_history = [w0]
    w = w0
    grad_f = grad(func)
    for k in range(n_steps):
        
        # Abstiegsrichtung
        d = -grad_f(w)

        # Stop sobald die Norm des Gradienten klein ist
        if np.linalg.norm(d) < 1e-6:
            break

        # Nächste Iterierte
        w = w + alpha * d
        
        w_history.append(w)

    return np.array(w_history)

# Initialisierung
w0 = np.zeros(5)
w_history = gd(func=binary_cross_entropy, alpha=0.01, w0=w0)

# Lösung ist letzte Iterierte
w_opt = w_history[-1]

print(f"Took {len(w_history)} iterations. Final objective is {binary_cross_entropy(w_opt)}")

Took 4750 iterations. Final objective is 45.30298215658562


### Konvexität der logistischen Regression

Als nächstes möchten wir überprüfen, ob ein Newton-Verfahren die Lösung schneller erreicht als das Gradientenverfahren. Dazu ist es hilfreich, zunächst zu überprüfen, ob die Hessematrix überall positiv (semi-)definit ist. Wenn dies nicht der Fall ist, so muss man Vorkehrungen treffen, die verhindern, dass das Newton Verfahren ein Maximum oder einen Sattelpunkt ansteuert. Zum Glück ist dies der Fall.

**Satz**

Die Hessematrix der binären Kreuzentropie für ein logistisches Regressionsmodell lautet
\begin{align*}
\nabla^2 L(\v w) = \v X^T \v D \v X,
\end{align*}
wobei $\v D$ eine Diagonalmatrix mit den Einträgen
\begin{align*}
\frac{1}{N}\sigma(\v w^T\v x_i)(1-\sigma(\v w^T\v x_i)),\quad i=1,\dots, N
\end{align*}
ist. Diese ist positiv semi-definit, daher ist das Minimierungsproblem konvex.

---

Dieses Resultat stellt sicher, dass die Newton-Richtung tatsächlich eine Abstiegsrichtung ist. Außerdem ist sichergestellt, dass jedes lokale Minimum auch ein globales Minimum ist. Wir benutzen die Methode `hessian` aus dem `autograd`-Paket zur automatischen Auswertung der Hessematrix. Außerdem benötigen wir die Funktion `solve` aus dem Paket `numpy.linalg`, um in jedem Schritt das Gleichungssystem zu lösen, welches uns die Newton-Richtung gibt. Obwohl das Ergebnis das gleiche ist, ist dies effizienter als die Inverse zu berechnen und die rechte Seite damit zu multiplizieren.

### Newton-Verfahren

In [6]:
from autograd import hessian
from numpy.linalg import solve

def newton(func, w0, n_steps=1000):
    """ Perform n_steps iterations of Newton's method with stepsize 1 and return iterates """
    w_history = [w0]
    w = w0
    grad_f = grad(func)
    hess_f = hessian(func)
    for k in range(n_steps):

        # Abstiegsrichtung
        g = grad_f(w)
        d = solve(hess_f(w), -g)
        
        # Stop if norm (length) of gradient vector is small
        if np.linalg.norm(g) < 1e-6:
            break

        # Next iterate
        w = w + d
        
        w_history.append(w)

    return np.array(w_history)

w0 = np.zeros(5)
w_history = newton(func=binary_cross_entropy, w0=w0)
w_opt = w_history[-1]

print(f"Took {len(w_history)} iterations. Final objective is {binary_cross_entropy(w_opt)}")

Took 6 iterations. Final objective is 45.30298215658388


Das Minimum ist nach 6 Iterationen erreicht. Wie erwartet stimmen der Zielfunktionswert der Lösung mit dem Gradientenverfahren und der Lösung, die das Newton-Verfahren produziert hat, überein. Die Koeffizienten $w_1,\dots,w_5$ quantifizieren den Einfluss der Features $x_1,\dots,x_5$ auf die Vorhersage.

In [7]:
print("Gewichte der Features:")
for i in range(5):
    print(f"{df.columns[i].ljust(20)}: {w_opt[i]: .3f}")

Gewichte der Features:
Age                 : -0.292
Gender              : -0.167
Annual Income       :  3.500
Browsing Time       : -1.245
Previous Purchases  : -0.103


Wir können das trainierte Modell außerdem benutzen um damit Vorhersagen zu machen. Dazu benutzen wir die logistische Funktion.

In [8]:
df["Purchased Prob"] = logistic(X, w_opt)
df["Purchased Pred"] = df["Purchased Prob"].round()
df["Vorhersage korrekt?"] = df["Purchased Pred"] == df["Purchased"]
print(f"Die Vorhersage ist in {df['Vorhersage korrekt?'].sum()} von {len(df)} Fällen korrekt.")

Die Vorhersage ist in 75 von 100 Fällen korrekt.


In [9]:
df

,Age,Gender,Annual Income,Browsing Time,Previous Purchases,Purchased,Purchased Prob,Purchased Pred,Vorhersage korrekt?
0,22,0,40,5,0,0,0.271966,0.0,True
1,35,1,60,10,3,1,0.502213,1.0,True
2,26,1,30,3,1,0,0.048725,0.0,True
3,29,0,55,8,2,1,0.614468,1.0,True
4,42,1,80,12,4,1,0.949208,1.0,True
...,...,...,...,...,...,...,...,...,...
95,24,1,34,3,1,0,0.116240,0.0,True
96,37,1,81,13,4,1,0.955447,1.0,True
97,28,0,40,5,1,0,0.196124,0.0,True
98,33,1,64,9,3,1,0.788221,1.0,True
